In [4]:
from acevedo_clss_and_fcns import *


def make_graphs_list(pyg_graph_in,mask_target:bool=False, target_node_1:int = -1, target_node_2:int = 1e10, mask_number = 1):
    
    pyg_graph = copy.deepcopy(pyg_graph_in)
    graphs_list = []
    
    
    for i in range(pyg_graph.x.shape[1]):
    
        
        new_pyg_data  = Data(x =  pyg_graph.x[:,i].reshape(pyg_graph.num_nodes, 1),  y = pyg_graph.y[i], 
                            edge_index = pyg_graph.edge_index)
        new_pyg_data.num_classes = 2
        if mask_target:
            new_pyg_data.x[target_node_1,:] = mask_number
            new_pyg_data.x[target_node_2,:] = mask_number        
        
        graphs_list.append(new_pyg_data) 
    return graphs_list

### Unmasked Phe

In [5]:
PYG_graph_Concen_plus_Fluxes       = torch.load("./results/graphs/PYG_graph_Concen_plus_Fluxes.pt") 
PYG_graph_only_Concen              = torch.load("./results/graphs/PYG_graph_only_Concen.pt")
graphs_list_PYG_graph_only_Concen  = make_graphs_list(PYG_graph_only_Concen)
graphs_list_PYG_Concen_plus_Fluxes = make_graphs_list(PYG_graph_Concen_plus_Fluxes)
loader_only_Concen                 = batch_loader(graphs_list_PYG_graph_only_Concen, batch_size= 4*32, validation_percent = .4)
loader_Concen_plus_Fluxes          = batch_loader(graphs_list_PYG_Concen_plus_Fluxes, batch_size= 4*32, validation_percent = .4)

torch.save(loader_only_Concen, "./results/dataloaders/loader_only_Concen.pt")
torch.save(loader_Concen_plus_Fluxes, "./results/dataloaders/loader_Concen_plus_Fluxes.pt")


### Masked Phe

In [6]:


grafo_nx_Concen_plus_Fluxes = nx.read_gpickle( "./results/graphs/NX_graph_Concen_plus_Fluxes.gpickle")

phe_L_c = list(grafo_nx_Concen_plus_Fluxes.nodes).index('phe_L_c')
tyr_L_c = list(grafo_nx_Concen_plus_Fluxes.nodes).index('tyr_L_c')

MASKED_only_Conc = make_graphs_list(PYG_graph_only_Concen,mask_target = True, 
                     target_node_1 = phe_L_c,
                     target_node_2 = tyr_L_c, 

                     mask_number = 1)

MASKED_Concen_plus_Fluxes = make_graphs_list(PYG_graph_Concen_plus_Fluxes,mask_target = True, 
                     target_node_1 = phe_L_c,
                     target_node_2 = tyr_L_c, 
                     mask_number = 1)


MASKED_loader_only_Concen                = batch_loader(MASKED_only_Conc, batch_size= 4*32, validation_percent = .4)
MASKED_loader_Concen_plus_Fluxes         = batch_loader(MASKED_Concen_plus_Fluxes, batch_size= 4*32, validation_percent = .4)


assert (1.0, 1.0) ==(next(iter(MASKED_loader_only_Concen.get_train_loader())).x.squeeze()[phe_L_c].item() , next(iter(MASKED_loader_only_Concen.get_train_loader())).x.squeeze()[tyr_L_c].item())
assert (1.0, 1.0) != (next(iter(loader_only_Concen.get_train_loader())).x.squeeze()[phe_L_c], next(iter(loader_Concen_plus_Fluxes.get_train_loader())).x.squeeze()[tyr_L_c])

torch.save(MASKED_loader_only_Concen, "./results/dataloaders/MASKED_loader_only_Concen.pt")
torch.save(MASKED_loader_Concen_plus_Fluxes, "./results/dataloaders/MASKED_loader_Concen_plus_Fluxes.pt")